In [1]:
%load_ext autoreload
%aimport -tf
%aimport data
%aimport model
%autoreload 1

In [3]:
import tensorflow as tf
import pandas as pd
import numpy as np
import data
import model

In [4]:
tf.logging.set_verbosity(tf.logging.INFO)

In [10]:
# Hyperparameters
MAX_DOCUMENT_LENGTH = 20
VOCAB_MIN_FREQUENCY = 100
DATA_TEST_SIZE = 5000

# Model Parameters
RNN_CELL_SIZE = 128
EMBEDDING_DIM = 64

# Training Parameters
TRAIN_BATCH_SIZE = 16
TRAIN_EVAL_EVERY = 1000

In [5]:
ds = data.load_reddit_data(
    max_document_length=MAX_DOCUMENT_LENGTH,
    min_frequency=VOCAB_MIN_FREQUENCY,
    test_size=DATA_TEST_SIZE)

In [6]:
data.print_dataset_stats(ds)

Vocabulary Size: 7794
Train Data Shape: (385367, 20)
Dev Data Shape (5000, 20)


In [7]:
train_input_fn, dev_input_fn = data.create_train_dev_input_fns(ds, batch_size=TRAIN_BATCH_SIZE)

In [8]:
def simple_rnn(sequences, sequence_lengths):
    cell = tf.nn.rnn_cell.GRUCell(RNN_CELL_SIZE)
    sequence_list = tf.unpack(sequences, axis=1)
    return tf.nn.rnn(cell, sequence_list, dtype=tf.float32, sequence_length=sequence_lengths)

model_fn = model.create_lm(
    vocab_size=len(ds.vocab.vocabulary_),
    embedding_dim=EMBEDDING_DIM,
    rnn_fn=simple_rnn)

In [11]:
# Monitor for sampling sentences from the estimator
sample_mon = model.SentenceSampleMonitor(
    vocab=ds.vocab,
    every_n_steps=TRAIN_EVAL_EVERY,
    first_n_steps=-1)

# Monitor for development set loss
dev_monitor = tf.contrib.learn.monitors.ValidationMonitor(
    input_fn=dev_input_fn,
    every_n_steps=TRAIN_EVAL_EVERY)

model_name = "maxlen_{}_rnn{}_embed_{}".format(MAX_DOCUMENT_LENGTH, RNN_CELL_SIZE, EMBEDDING_DIM)
estimator = tf.contrib.learn.Estimator(model_fn=model_fn, model_dir="./checkpoints/{}".format(model_name))

In [12]:
estimator.fit(input_fn=train_input_fn, steps=None, monitors=[sample_mon, dev_monitor])

INFO:tensorflow:Create CheckpointSaver
INFO:tensorflow:Step 1: loss = 4.06178
INFO:tensorflow:Step 101: loss = 2.6804
INFO:tensorflow:Step 201: loss = 3.89922
INFO:tensorflow:Saving checkpoints for 300 into ./checkpoints/maxlen_20_rnn128_embed_64/model.ckpt.
INFO:tensorflow:Step 301: loss = 1.7498
INFO:tensorflow:Step 401: loss = 1.92708
INFO:tensorflow:Step 501: loss = 2.93516
INFO:tensorflow:Saving checkpoints for 600 into ./checkpoints/maxlen_20_rnn128_embed_64/model.ckpt.
INFO:tensorflow:Step 601: loss = 2.18486
INFO:tensorflow:Step 701: loss = 2.51499
INFO:tensorflow:Step 801: loss = 2.24487
INFO:tensorflow:Saving checkpoints for 900 into ./checkpoints/maxlen_20_rnn128_embed_64/model.ckpt.
INFO:tensorflow:Step 901: loss = 1.75796


Sampling from model at ./checkpoints/maxlen_20_rnn128_embed_64/model.ckpt-900-?????-of-00001


INFO:tensorflow:Error reported to Coordinator: <class 'ValueError'>, Cannot feed value of shape (1, 20) for Tensor 'Placeholder:0', which has shape '(?, 40)'


SETENCE_START


ValueError: Cannot feed value of shape (1, 20) for Tensor 'Placeholder:0', which has shape '(?, 40)'